# Worksheet 8.2: Introduction to ROS Programming (Hello World)

This worksheet takes you through the basics of setting up your workspace and creating a simple package in ROS2. This is not an executable file and you need to run them as a script.

By completing this worksheet, you will be better able to:

- **explain** the purpose and function of the Robot Operating System (ROS)
- **describe** how “messages”, “topics”, and “message types” work in ROS
- **demonstrate** skills to use ROS messages to make two ROS processes communicate

**NOTE**
We assume you have basic familiarity with the Linux command line environment. If you have never used commands like `cd` or `ls` or `mkdir` before, then complete the first four steps in this <a href="https://ubuntu.com/tutorials/command-line-for-beginners" target="_blank">ubuntu tutorial for beginners</a> before you continue.


**NOTE THE WORKSPACE IS ALREADY CREATED FOR YOU IN THE VM PROVIDED**


**NOTE: This notebook is not an interactive one. You need to run the code in the form of scripts.**

## TASK1: Create Your Workspace

First, create a directory ```(ros2_ws)``` to contain our workspace:

In [ ]:
mkdir -p ~/ros2_ws/src
cd ~/ros2_ws

## TASK2: Create your package

Open a new terminal and source your ROS 2 installation so that ros2 commands will work.

Navigate into the ```ros2_ws``` directory.

Recall that packages should be created in the src directory, not the root of the workspace. So, navigate into ros2_ws/src, and run the package creation command:

In [ ]:
ros2 pkg create --build-type ament_python unit2

**Build a package**

In [ ]:
cd ~/ros2_ws   # go to root directory

colcon build   # build your package

source install/local_setup.bash   # source

The package has been created successfully. 

# TASK3: Write a simple subscriber and publisher script

The idea is to write and understand simple code for subscriber and publisher which will help us understand different topics and msgs type. Once, we complete this task, we will move towards the turtlebot3 to do the same.

Navigate to ```ros2_ws/src/unit2/unit2```. Download the example talker (Publisher) code by entering the following command:

In [ ]:
wget https://raw.githubusercontent.com/ros2/examples/foxy/rclpy/topics/minimal_publisher/examples_rclpy_minimal_publisher/publisher_member_function.py

Now there will be a new file named ```publisher_member_function.py``` adjacent to ```__init__.py```. The code looks like this

In [ ]:
import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalPublisher(Node):

    def __init__(self):
        super().__init__('minimal_publisher')
        self.publisher_ = self.create_publisher(String, 'topic', 10)
        timer_period = 0.5  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.i = 0

    def timer_callback(self):
        msg = String()
        msg.data = 'Hello World: %d' % self.i
        self.publisher_.publish(msg)
        self.get_logger().info('Publishing: "%s"' % msg.data)
        self.i += 1


def main(args=None):
    rclpy.init(args=args)

    minimal_publisher = MinimalPublisher()

    rclpy.spin(minimal_publisher)

    # Destroy the node explicitly
    # (optional - otherwise it will be done automatically
    # when the garbage collector destroys the node object)
    minimal_publisher.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

## Examine the code

The first lines of code after the comments ```import rclpy``` so its Node class can be used.

In [ ]:
import rclpy
from rclpy.node import Node

The next statement imports the built-in string message type that the node uses to structure the data that it passes on the topic.

In [ ]:
from std_msgs.msg import String

These lines represent the node’s dependencies. Recall that dependencies have to be added to package.xml, which you’ll do in the next section.

Next, the MinimalPublisher class is created, which inherits from (or is a subclass of) Node.

In [ ]:
class MinimalPublisher(Node):

Following is the definition of the class’s constructor. super().__init__ calls the Node class’s constructor and gives it your node name, in this case minimal_publisher.

create_publisher declares that the node publishes messages of type String (imported from the std_msgs.msg module), over a topic named topic, and that the “queue size” is 10. Queue size is a required QoS (quality of service) setting that limits the amount of queued messages if a subscriber is not receiving them fast enough.

Next, a timer is created with a callback to execute every 0.5 seconds. self.i is a counter used in the callback.

In [ ]:
def __init__(self):
    super().__init__('minimal_publisher')
    self.publisher_ = self.create_publisher(String, 'topic', 10)
    timer_period = 0.5  # seconds
    self.timer = self.create_timer(timer_period, self.timer_callback)
    self.i = 0

timer_callback creates a message with the counter value appended, and publishes it to the console with get_logger().info.

In [ ]:
def timer_callback(self):
    msg = String()
    msg.data = 'Hello World: %d' % self.i
    self.publisher_.publish(msg)
    self.get_logger().info('Publishing: "%s"' % msg.data)
    self.i += 1

Lastly, the main function is defined.

In [ ]:
def main(args=None):
    rclpy.init(args=args)

    minimal_publisher = MinimalPublisher()

    rclpy.spin(minimal_publisher)

    # Destroy the node explicitly
    # (optional - otherwise it will be done automatically
    # when the garbage collector destroys the node object)
    minimal_publisher.destroy_node()
    rclpy.shutdown()

First the rclpy library is initialized, then the node is created, and then it “spins” the node so its callbacks are called.

## ADD dependencies and add the entry point

Please add these dependencies to package.xml and add the script to console_scripts brackets of the entery_points field in setup.py

In [ ]:
<exec_depend>rclpy</exec_depend>
<exec_depend>std_msgs</exec_depend>

In [ ]:
entry_points={
        'console_scripts': [
                'talker = unit2.publisher_member_function:main',
        ],
},

## Write the listener Node (subscriber)

Go to ```ros2_ws/src/unit2/unit2``` and download the subscriber code

In [ ]:
wget https://raw.githubusercontent.com/ros2/examples/foxy/rclpy/topics/minimal_subscriber/examples_rclpy_minimal_subscriber/subscriber_member_function.py

The directory should have these files now

In [ ]:
__init__.py  publisher_member_function.py  subscriber_member_function.py

## Add an entry point

Reopen setup.py and add the entry point for the subscriber node below the publisher’s entry point. The entry_points field should now look like this

In [ ]:
entry_points={
        'console_scripts': [
                'talker = unit2.publisher_member_function:main',
                'listener = unit2.subscriber_member_function:main',
        ],
},

## Build and run

Check for dependencies 

In [ ]:
rosdep install -i --from-path src --rosdistro foxy -y

In [ ]:
colcon build --packages-select unit2

source install/setup.bash

## Run your code

Now, your both subscriber and publisher nodes are ready. Open a new terminal and source it. Run this command after that

In [ ]:
ros2 run unit2 talker

The terminal should start publishing info messages every 0.5 seconds, like so:

In [ ]:
[INFO] [minimal_publisher]: Publishing: "Hello World: 0"
[INFO] [minimal_publisher]: Publishing: "Hello World: 1"
[INFO] [minimal_publisher]: Publishing: "Hello World: 2"
[INFO] [minimal_publisher]: Publishing: "Hello World: 3"
[INFO] [minimal_publisher]: Publishing: "Hello World: 4"
...

Open another terminal, source the setup files from inside ros2_ws again, and then start the listener node:

In [ ]:
ros2 run unit2 listener

The listener will start printing messages to the console, starting at whatever message count the publisher is on at that time, like so:

In [ ]:
[INFO] [minimal_subscriber]: I heard: "Hello World: 10"
[INFO] [minimal_subscriber]: I heard: "Hello World: 11"
[INFO] [minimal_subscriber]: I heard: "Hello World: 12"
[INFO] [minimal_subscriber]: I heard: "Hello World: 13"
[INFO] [minimal_subscriber]: I heard: "Hello World: 14"

## Subscriber
The subscriber has a similar structure to the publisher but it is simpler. There is no timer() because it will be invoked by the topic it subscribes to. Note also that the call_back function expects an argument msg which is the info sent by the topic the subscriber subscribes to.


In [ ]:
import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalSubscriber(Node):

    def __init__(self):
        super().__init__('minimal_subscriber')
        self.subscription = self.create_subscription(String, 'topic', self.listener_callback, 10)
        self.subscription  # prevent unused variable warning

    def listener_callback(self, msg):
        self.get_logger().info('I heard: "%s"' % msg.data)


def main(args=None):
    rclpy.init(args=args)

    minimal_subscriber = MinimalSubscriber()

    rclpy.spin(minimal_subscriber)

    # Destroy the node explicitly
    # (optional - otherwise it will be done automatically
    # when the garbage collector destroys the node object)
    minimal_subscriber.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()

Now run the scubscriber and observe what its output, what do you conclude?

## Summary

You created two nodes to publish and subscribe to data over a topic. Before running them, you added their dependencies and entry points to the package configuration files.